# Dataset Preparation

Use the `COCO API` to create images of Segmentation Masks for the class: `person`

In [1]:
from pycocotools.coco import COCO
import os
import cv2
import time
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import random

In [2]:
# Annotation Files
train_ann = "/media/ActiveTraining/Datasets/COCO/annotations/instances_train2017.json"
val_ann = "/media/ActiveTraining/Datasets/COCO/annotations/instances_val2017.json"

train_img_dir = '/media/ActiveTraining/Datasets/COCO/train/'
val_img_dir = '/media/ActiveTraining/Datasets/COCO/val/'

train_dest_segmentation_masks = '/media/ActiveTraining/Datasets/COCO/segmentations/train'
val_dest_segmentation_masks = '/media/ActiveTraining/Datasets/COCO/segmentations/val'

### Load the COCO Dataset

In [3]:
# Initialize the COCO objects
train_coco = COCO(train_ann)
val_coco = COCO(val_ann)

loading annotations into memory...
Done (t=15.27s)
creating index...
index created!
loading annotations into memory...
Done (t=0.61s)
creating index...
index created!


### Functions to Convert COCO to Segmentation Masks

In [4]:
def get_mask(coco_object, index, filtered_cat_id, filtered_img_id, img_dir):
    # Load Image
    img_meta = coco_object.loadImgs(filtered_img_id[index])
    img_path = os.path.join(img_dir, img_meta[0]['file_name'])
    img = cv2.imread(img_path)

    # Load Annotations
    annIds = coco_object.getAnnIds(imgIds=img_meta[0]['id'], catIds=filtered_cat_id, iscrowd=None)
    anns = coco_object.loadAnns(annIds)
    
    # Generate Mask
    anns_img = np.zeros((img.shape[:-1]), dtype=np.uint8)
    for ann in anns:
        anns_img = np.maximum(anns_img, coco_object.annToMask(ann)*ann['category_id'])
        
    return img_meta[0]['file_name'], anns_img * 255


def write_mask(coco_object, index, filtered_cat_id, filtered_img_id, dest_folder, img_dir):
    if not os.path.isdir(dest_folder):
        os.makedirs(dest_folder)
        
    filename, mask = get_mask(coco_object, index, filtered_cat_id, filtered_img_id, img_dir)
    dest_path = os.path.join(dest_folder, filename)
    cv2.imwrite(dest_path, mask)


def write_train_masks(train_coco, train_dest_folder, classes=['person']):
    print("Processing Train Images")
    time.sleep(0.5)
    filtered_cat_id = train_coco.getCatIds(classes)
    filtered_img_id = train_coco.getImgIds(catIds=filtered_cat_id) 
    
    for i in tqdm(range(len(filtered_img_id))):
        write_mask(train_coco, i, filtered_cat_id, filtered_img_id, train_dest_folder, train_img_dir)
        
    print("Train Segmentation Masks Processing Complete")


def write_val_masks(val_coco, val_dest_folder, classes=['person']):
    print("Processing Val Images")
    time.sleep(0.5)
    filtered_cat_id = val_coco.getCatIds(classes)
    filtered_img_id = val_coco.getImgIds(catIds=filtered_cat_id) 
    
    for i in tqdm(range(len(filtered_img_id))):
        write_mask(val_coco, i, filtered_cat_id, filtered_img_id, val_dest_folder, val_img_dir)
        
    print("Val Segmentation Masks Processing Complete")


In [5]:
write_train_masks(train_coco, train_dest_segmentation_masks)
write_val_masks(val_coco, val_dest_segmentation_masks)

Processing Train Images


100%|██████████| 64115/64115 [07:55<00:00, 134.84it/s]


Train Segmentation Masks Processing Complete
Processing Val Images


100%|██████████| 2693/2693 [00:20<00:00, 132.84it/s]

Val Segmentation Masks Processing Complete
